# Streams

In [3]:
*standard-input*
*standard-output*

#<JUPYTER::STDIN-STREAM {1100BA00A3}>

#<JUPYTER::IOPUB-STREAM {1100BA0093}>

In [4]:
; pathname
(setf path (make-pathname :name "myfile"))
; open
(setf str (open path :direction :output
                :if-exists :supersede))

(format str "Something~%")

; close
(close str)

#P"myfile"

#<SB-SYS:FD-STREAM for "file d:\\GoogleDrive\\code\\jupyter-notebooks\\CommonLisp\\books\\ANSI Common Lisp\\myfile" {1102AD2823}>

NIL

T

SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::PATH
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::PATH
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::STR
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::STR
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::STR


In [5]:
; read file
(setf str (open path :direction :input))
(read-line str)

(close str)

#<SB-SYS:FD-STREAM for "file d:\\GoogleDrive\\code\\jupyter-notebooks\\CommonLisp\\books\\ANSI Common Lisp\\myfile" {1102B8B6A3}>

"Something"

NIL

T

SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::PATH
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::STR
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::STR
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::STR


In [6]:
; delete-file
(delete-file path)

T

SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::PATH


In [8]:
; with-open-file
(with-open-file (str path :direction :output
                     :if-exists :supersede)
  (format str "Something~%"))

(with-open-file (str path :direction :input)
  (read-line str))

(delete-file path)

NIL

"Something"

NIL

T

SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::PATH
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::PATH
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::PATH


# Input

In [9]:
; read-line
(progn
 (format t "Please enter your name: ")
 (read-line))

Please enter your name: 

"Rodrigo de Bivar"

NIL

In [10]:
(defun pseudo-cat (file)
  (with-open-file (str file :direction :input)
    (do ((line (read-line str nil 'eof)
               (read-line str nil 'eof)))
        ((eql line 'eof))
      (format t "~A~%" line))))

(pseudo-cat (make-pathname :name "sample.txt"))

PSEUDO-CAT

NIL

The quick brown fox jumps over the lazy dog.


In [11]:
; read
(read)

(A B C)

In [12]:
; read-from-string
(read-from-string "a b c")

A

2

# Output

In [14]:
; prin1: output for programs
; princ: output for people
(prin1 "Hello")
(princ "Hello")

"Hello"

"Hello"

"Hello"Hello

In [ ]:
; format
; ~A: as princ
(format nil "Dear ~A,~% Our records indicate..."
  "Mir. Malatesta")

"Dear Mir. Malatesta,
 Our records indicate..."

In [17]:
; ~S: as prin1
(format t "~S ~A" "z" "z")

NIL

"z" z

In [20]:
; ~F
(format nil "~10,2,0,'*,' F" 26.21875)
(format nil "~,2,,,F" 26.21875) 
(format nil "~,2F" 26.21875) 

"     26.22"

"26.22"

"26.22"

# Example: String Substitution

In [ ]:
; a ring buffer
(defstruct buf
  vec
  (start -1)
  (used -1)
  (new -1)
  (end -1))

; return the element at index n in buf
(defun bref (buf n)
  (svref (buf-vec buf)
         (mod n (length (buf-vec buf)))))

(defun (setf bref) (val buf n)
  (setf (svref (buf-vec buf)
               (mod n (length (buf-vec buf))))
    val))

(defun new-buf (len)
  (make-buf :vec (make-array len)))

(defun buf-insert (x b)
  (setf (bref b (incf (buf-end b))) x))

(defun buf-pop (b)
  (prog1
      (bref b (incf (buf-start b)))
    (setf (buf-used b) (buf-start b)
      (buf-new b) (buf-end b))))

(defun buf-next (b)
  (when (< (buf-used b) (buf-new b))
        (bref b (incf (buf-used b)))))

(defun buf-reset (b)
  (setf (buf-used b) (buf-start b)
    (buf-new b) (buf-end b)))

(defun buf-clear (b)
  (setf (buf-used b) -1 (buf-start b) -1
    (buf-new b) -1 (buf-end b) -1))

(defun buf-flush (b str)
  (do ((i (1+ (buf-used b)) (1+ i)))
      ((> i (buf-end b)))
    (princ (bref b i) str)))

; substitute
(defun stream-subst (old new in out)
  (let* ((pos 0)
         (len (length old))
         (buf (new-buf len))
         (from-buf nil))
    (do ((c (read-char in nil :eof)
            (or (setf from-buf (buf-next buf))
                (read-char in nil :eof))))
        ((eql c :eof))
      (cond ((char= c (char old pos))
              (incf pos)
              (cond ((= pos len) ; 3
                                (princ new out)
                                (setf pos 0)
                                (buf-clear buf))
                    ((not from-buf) ; 2
                                   (buf-insert c buf))))
            ((zerop pos) ; 1
                        (princ c out)
                        (when from-buf
                              (buf-pop buf)
                              (buf-reset buf)))
            (t ; 4
              (unless from-buf
                (buf-insert c buf))
              (princ (buf-pop buf) out)
              (buf-reset buf)
              (setf pos 0))))
    (buf-flush buf out)))

(defun file-subst (old new file1 file2)
  (with-open-file (in file1 :direction :input)
    (with-open-file (out file2 :direction :output
                         :if-exists :supersede)
      (stream-subst old new in out))))

BUF

BREF

(SETF BREF)

NEW-BUF

BUF-INSERT

...

BUF-RESET

BUF-CLEAR

BUF-FLUSH

STREAM-SUBST

FILE-SUBST

In [2]:
 (file-subst " th" " z" "test1" "test2")

NIL

# Macro Characters

In [3]:
; macro character: get special treatment from `read`
; read-macro: a macro character, combination of macro characters

(car (read-from-string "'a"))

QUOTE

In [4]:
; dispatching read-macro
; example: 
; #': (function ...)
; #(...): vector
; #nA(...): array 
; #\,: character
; #S(n ...): structure

(prin1 (vector 1 2))
(let ((*print-array* t))
  (vectorp (read-from-string (format nil "~S" (vector 1 2)))))

#(1 2)

T

#(1 2)

In [6]:
(prin1 #'prin1)
(prin1 (make-hash-table))

#<FUNCTION PRIN1>

#<HASH-TABLE :TEST EQL :COUNT 0 {1102E82553}>

#<FUNCTION PRIN1>#<HASH-TABLE :TEST EQL :COUNT 0 {1102E82553}>